In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

from tqdm import trange, tqdm

sys.path.append('../..')
from svm import svm as SVM

import scipy
from statsmodels.stats.multitest import multipletests

#
import pandas as pd
import matplotlib.patches as mpatches
import scipy.stats as stats


Autosaving every 180 seconds


<ipython-input-1-2ad95b854a16>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# #######################################################
# #### RERUN SVM BUT LOWPASS FILTER LOCANMF COMPS #######
# #######################################################

#
def load_supersessions(animal_id, lockout):
    fname = os.path.join('/media/cat/4TBSSD/yuki/',
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    trials = np.vstack(data['trials'])
    random = np.vstack(data['random'])
    
    return trials, random

# 
def filter_trace_box_convolve(trace, smooth_window=30):

    box = np.ones(smooth_window)/smooth_window
    trace_smooth = np.convolve(trace, box, mode='same')

    return trace_smooth

from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y


def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)
        sig.append(res[1])

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    #print ("sig (BH corrected):", sig)
    edt = sig.shape[0]
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec


In [12]:
############################################
########## QUICK AND DIRTY CHECKS ##########
############################################
#session_name = 'IA1pm_Feb23_30Hz'
fnames = [

    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_400_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_350_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_300_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_250_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_150_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_100_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_50_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz'

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
    
    # IA1 15sec lockouts
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_100_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_300_super_res.npz',
    
    # IA2
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_400_super_res.npz'
    
    # IA3
    #'/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    
    # IJ1
    #'/media/cat/4TBSSD/yuki/IJ1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',

    # AQ2
    #'/media/cat/4TBSSD/yuki/AQ2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    
    
    
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_100_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz'
    '/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_400_super_res.npz'
    
    
]

# 
fig=plt.figure(figsize=(10,10))
ax=plt.subplot(111)
for ctr,fname in enumerate(fnames):
    #
    data = np.load(fname)
        
    #
    predictions = data['predictions']
    print ("n_trials: ", predictions.shape[1])
    acc = data['accuracy']
    #length = 1801-acc.shape[0]
    #acc = np.vstack(
    #                 (np.zeros((length,10), dtype=np.float32), 
    #                  acc))
    
    #acc2 = []
    for k in range(acc.shape[1]):
        #acc[:,k] = butter_lowpass_filter(acc[:,k],0.5, 30)    
        acc[:,k] = filter_trace_box_convolve(acc[:,k])
            
    print (acc.shape)
    t = np.arange(acc.shape[0])/30-29
    mean = acc.mean(1)
    std = np.std(acc,1)
    plt.plot(t,mean, 
             c='blue',
             linewidth=10,
             label=fname)
    ax.fill_between(t, mean+std,
                    mean-std,
                    color='blue',
                    alpha=.2)

#
plt.plot([-30,30],[0.5,0.5],'--',c='grey')
plt.ylim(0,1)
plt.xlim(-20,0)
#plt.legend(fontsize=6)
if False:
    plt.savefig('/home/cat/acc_filtered_IA1.svg')
    plt.close()
else:
    plt.show()



n_trials:  328
(900, 10)


In [42]:
fname = '/media/cat/4TBSSD/yuki/IA1/first_decoding_time_0.3hzFilter.npz'
data=np.load(fname)

sessions = data['all_session_names']
print (sessions.shape)

(53,)
